## Setup

In [114]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual, train_test_to_df
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

In [115]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"], "DEPTH": PARAMETERS["OUTLIER_NOISE"]["DEPTH"], "MULTI": PARAMETERS["OUTLIER_NOISE"]["MULTI"] }

FEATURE_SEL_CORR = {"THRESHOLD_CORR": PARAMETERS["FEATURE_SEL_CORR"]["THRESHOLD"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [116]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [117]:
df = dropMissingValues(df) 

## Outlier Detection

In [118]:
# Outlier Detection
outlier_dict_all = {}

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    df = drop99_all(df, outlier_list_99 )
    #outlier_dict_all["99_mean"] = outlier_list_99
    #outlier_dict_all["99_drop"] = outlier_list_99
    print(green('done'), "\n")

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

if OUTLIER_ZSCORE_IND == True:
    outlier_list_z_score_ind = z_score_individual(df)

print(green('done'), '\n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print(green('done'), '\n\nstarting db_scan')
for k in OUTLIER_DBSCAN["K"]:
    for o in OUTLIER_DBSCAN["X"]:
        for e in OUTLIER_DBSCAN["EPS"]:
            for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                print(k, o, e, m)
                outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                outlier_dict_all["db_scan_"+str(k)+"_"+str(o)+"_"+str(e)+"_"+str(m)] = outlier_list_dbscan

print(green('done'), '\n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
    for d in OUTLIER_NOISE["DEPTH"]:
        for m in OUTLIER_NOISE["MULTI"]:
            print(cv, d, m)
            outlier_list_noise = getNoise(df, cv, d, m)
            outlier_dict_all["noise_"+str(cv)+"_"+str(d)+"_"+str(m)] = outlier_list_noise
print(green('done'))

outlier_99
done 

starting z_score
10.1
done 

starting knn
3 1500
done 

starting db_scan
4 500 0.6 8
4 500 0.4 8
done 

starting noise
5 5 10
done


## Combination

In [119]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

...
Combination:  ('knn_3_1500', 'db_scan_4_500_0.4_8') 

Combination:  ('knn_3_1500', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.4_8') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

## Algorythm Selection

In [120]:
model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        
        # if "99_mean" in c:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
        # else:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        #df_price_rem = df.drop(columns=["price"])
        # print(df_price_rem[:2])
        #print(df[:1])
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop") #TODO drop entfernen

        #TODO Regressionmodels
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
        model_obj_list.append(obj)

    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 53 rows

1: ('knn_3_1500',)
dropped 1500 / 1500 rows

2: ('db_scan_4_500_0.6_8',)
dropped 525 / 525 rows

3: ('db_scan_4_500_0.4_8',)
dropped 3783 / 3783 rows

4: ('noise_5_5_10',)
dropped 35 / 51 rows

5: ('z_score_10.1', 'knn_3_1500')
dropped 1536 / 1536 rows

6: ('z_score_10.1', 'db_scan_4_500_0.6_8')
dropped 562 / 562 rows

7: ('z_score_10.1', 'db_scan_4_500_0.4_8')
dropped 3814 / 3814 rows

8: ('z_score_10.1', 'noise_5_5_10')
dropped 76 / 76 rows

9: ('knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1534 / 1534 rows

10: ('knn_3_1500', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

11: ('knn_3_1500', 'noise_5_5_10')
dropped 1532 / 1532 rows

12: ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

13: ('db_scan_4_500_0.6_8', 'noise_5_5_10')
dropped 559 / 559 rows

14: ('db_scan_4_500_0.4_8', 'noise_5_5_10')
dropped 3808 / 3808 rows

15: ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1569 / 1569 rows

16: ('z_score

In [121]:
#model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        df_lasso = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        X_l, y_l = df_lasso.drop(columns=["price", "date", "id"]), df_lasso["price"]
        scaler = StandardScaler().fit(X_l)
        X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size = SPLIT_SIZE, random_state=1) #TODO 
        #rint(X_train_l)
        X_train_l = scaler.transform(X_train_l)
        X_test_l = scaler.transform(X_test_l)
        
        reg = Lasso(alpha=1)
        obj_l = Model(reg, (X_train_l, X_test_l, y_train_l, y_test_l), c, df_lasso, X_l.columns.to_list())
        model_obj_list.append(obj_l)
        ''''''
    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 41 rows

1: ('knn_3_1500',)
dropped 1500 / 1500 rows

2: ('db_scan_4_500_0.6_8',)
dropped 525 / 525 rows

3: ('db_scan_4_500_0.4_8',)
dropped 3783 / 3783 rows

4: ('noise_5_5_10',)
dropped 35 / 35 rows

5: ('z_score_10.1', 'knn_3_1500')
dropped 1536 / 1536 rows

6: ('z_score_10.1', 'db_scan_4_500_0.6_8')
dropped 562 / 562 rows

7: ('z_score_10.1', 'db_scan_4_500_0.4_8')
dropped 3814 / 3814 rows

8: ('z_score_10.1', 'noise_5_5_10')
dropped 76 / 76 rows

9: ('knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1534 / 1534 rows

10: ('knn_3_1500', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

11: ('knn_3_1500', 'noise_5_5_10')
dropped 1532 / 1532 rows

12: ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

13: ('db_scan_4_500_0.6_8', 'noise_5_5_10')
dropped 559 / 559 rows

14: ('db_scan_4_500_0.4_8', 'noise_5_5_10')
dropped 3808 / 3808 rows

15: ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1569 / 1569 rows

16: ('z_score

In [122]:
df_lasso.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,dis_super,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,ahf1,ahf2,ahf3,price
0,3.438501e+09,1.418602e+18,3.0,1.00,970.0,6828.0,1.0,0.0,640.05,0.0,...,0.0,98106.0,47.5476,-122.360,1160.0,11666.0,39.376922,114.928298,154.305220,315000.0
1,2.598010e+08,1.425859e+18,4.0,2.00,1610.0,8000.0,1.0,0.0,739.03,0.0,...,0.0,98008.0,47.6301,-122.118,1560.0,7896.0,82.463087,97.001643,179.464731,526000.0
2,3.832060e+09,1.426464e+18,4.0,2.50,2200.0,5893.0,2.0,0.0,501.91,0.0,...,0.0,98042.0,47.3333,-122.055,2200.0,5757.0,58.893575,119.451791,178.345367,280000.0
3,1.722069e+09,1.419811e+18,3.0,2.50,3100.0,100188.0,1.0,0.0,909.06,0.0,...,0.0,98038.0,47.3928,-122.066,2430.0,104979.0,66.186721,109.383269,175.569991,540000.0
4,9.268850e+09,1.428624e+18,3.0,1.75,1300.0,1237.0,2.0,0.0,479.66,0.0,...,0.0,98027.0,47.5390,-122.026,1350.0,942.0,64.025540,99.892763,163.918304,308000.0


In [123]:
#Summary
df_summary = pd.DataFrame(columns=['model', 'combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'model': o.get_model(), 'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,model,combo,mae,score
60,Lasso(alpha=1),"(knn_3_1500, db_scan_4_500_0.6_8, db_scan_4_50...",100320.58,0.677609
45,Lasso(alpha=1),"(db_scan_4_500_0.4_8, noise_5_5_10)",100320.58,0.677609
54,Lasso(alpha=1),"(knn_3_1500, db_scan_4_500_0.4_8, noise_5_5_10)",100320.58,0.677609
55,Lasso(alpha=1),"(db_scan_4_500_0.6_8, db_scan_4_500_0.4_8, noi...",100320.58,0.677609
61,Lasso(alpha=1),"(z_score_10.1, knn_3_1500, db_scan_4_500_0.6_8...",103119.48,0.692954
...,...,...,...,...
10,LinearRegression(),"(knn_3_1500, db_scan_4_500_0.4_8)",162867.97,0.431331
3,LinearRegression(),"(db_scan_4_500_0.4_8,)",162867.97,0.431331
0,LinearRegression(),"(z_score_10.1,)",165149.5,0.525593
8,LinearRegression(),"(z_score_10.1, noise_5_5_10)",165189.16,0.525594


In [124]:
number = 15
print(model_obj_list[number])
lassotest = model_obj_list[number]
#print(lassotest.get_data()[0].shape, lassotest.get_data()[1].shape , lassotest.get_data()[2].shape, lassotest.get_data()[3].shape )


Model: LinearRegression() Combination: ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8') Features: 25


In [125]:
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#score_best = df_summary[df_summary.score==df_summary.score.max()]
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)
#rint(f'{bold("Lowest mae:")} {mae_best}')

Model: Lasso(alpha=1) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 22


## Feature Selection

In [126]:
model_obj_list_best = [best_model_obj]
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_feature_reduction = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj.get_features())
#print(df_feature_reduction)

for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
    #print(threshold, type(threshold))
    try: 
        rel_features = getRelFeatures(df_feature_reduction, threshold)
        #print(len(rel_features), rel_features)
        df_feature_reduction = df_feature_reduction[rel_features]
        X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
        best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction, df_feature_reduction.columns.to_list())
        model_obj_list_best.append(best_model_obj_new)

    except Exception as e:
        print(bold("Error"), red(e))

##manual feature selection
rel_features_m = []

print(model_obj_list_best)

[Model: Lasso(alpha=1) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 23, Model: Lasso(alpha=1) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 10, Model: Lasso(alpha=1) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 9, Model: Lasso(alpha=1) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 9]


In [127]:
#Summary
df_summary_best = pd.DataFrame(columns=['model', 'combo', 'mae', 'score', 'features'])
for o in model_obj_list_best:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    features = o.get_features()
    if "price" in features:
        features.remove("price")
    row = {'model': o.get_model(),'combo': comb, 'mae': mae, 'score': score, 'features': len(features)}
    df_summary_best = df_summary_best.append(row, ignore_index=True)

df_summary_best.sort_values("mae")

,model,combo,mae,score,features
0,Lasso(alpha=1),"(db_scan_4_500_0.4_8, noise_5_5_10)",100320.58,0.677609,22
1,Lasso(alpha=1),"(db_scan_4_500_0.4_8, noise_5_5_10)",109365.88,0.642075,9
2,Lasso(alpha=1),"(db_scan_4_500_0.4_8, noise_5_5_10)",113435.67,0.592567,8
3,Lasso(alpha=1),"(db_scan_4_500_0.4_8, noise_5_5_10)",113435.67,0.592567,8


In [128]:
# #row
# mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
# #score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
# best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
# print(best_model_obj)
# print(f'{bold("Lowest mae:")} {mae_best}')


In [129]:
# number = 15
# print(model_obj_list[number])
# lassotest = model_obj_list[number]
# X_train, X_test, y_train, y_test =  lassotest.get_data()
# X = np.append(X_train, X_test, axis=0)
# print(type(y_test))
# y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
# print(X_train.shape, X_test.shape)
# print(y_train.shape, y_test.shape)

# X_y = np.append(X, y, axis=1)
# features = df.columns.to_list()
# df_adsa = pd.DataFrame(X_y, columns=features)

# df_adsa

In [130]:
# def train_test_to_df(X_train, X_test, y_train, y_test, columns):
#     X = np.append(X_train, X_test, axis=0)
#     columns.remove("price")
#     columns.append("price")
#     #print(X_train.shape)
#     print("X", X.shape)
#     try:
#         y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)

#     except:
#         y = np.append(y_train, y_test, axis=0)
#         y = y.reshape(len(y), 1)
        
#     print("y", y.shape)
#     X_y = np.append(X, y, axis=1)
#     print("X_y", X_y.shape)
#     #features = df.columns.to_list()
#     df = pd.DataFrame(X_y, columns=columns)
#     return df

In [131]:
# def train_test_to_df(X_train, X_test, y_train, y_test, columns):
#     X_train = pd.DataFrame(X_train, columns = df.columns.to_list().remove("price"))
#     X_test = pd.DataFrame(X_test, columns = df.columns.to_list().remove("price"))
#     y_train = pd.DataFrame(y_train, columns  = ["price"])
#     y_test = pd.DataFrame(y_test, columns = ["price"])

    
#     return df

In [132]:
# #print(df.columns.to_list())
# X_train, X_test, y_train, y_test =  lassotest.get_data()

# columnsList = lassotest.get_features()
# #columnsList.remove("price")

# #columnsList.append("price")
# print(X_train.shape, X_test.shape,y_train.shape, y_test.shape)
# train_test_to_df(X_train, X_test, y_train, y_test, columnsList)

## Regressionmodels

In [133]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=2)
# X_train, X_test, y_train, y_test = best_model_obj_new.get_data()
# dff = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj_new.get_features())
# poly.fit_transform(dff)
# obj_l = Model(poly, best_model_obj_new.get_data(), best_model_obj_new.get_comb(), best_model_obj_new.get_df(), best_model_obj_new.get_features())
# o = obj_l.train()
# mae, score = obj_l.summary()

## Evaluatuion

## Testdata

In [134]:
# combination_list = []
# df_feature_reduction = best_model_obj.get_df()

# for i, combo in enumerate(getCombinations(df_feature_reduction.columns.to_list()), 1):
#     if not i == 1: #emtpy tuple
#         combination_list.append(combo)
#         # print('combo #{}: {}'.format(i, combo))
# print("...")
# for c in combination_list[10:]:
#     print(bold("Combination: "), blue(c),"\n",)
# print(bold(len(combination_list)), " total Combinations")

In [135]:
# model_obj_list_best = []
# df_feature_reduction = best_model_obj.get_df()
# for i, c in enumerate(combination_list):
#     print(f'\n{bold(i)}: {blue(c)}')
    
#     try: 
#         X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#         best_model_obj_copy = best_model_obj.copy()
#         best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

#         model_obj_list_best.append(obj)

#     except Exception as e:
#         print(bold("Error"), red(e))

In [136]:
# df_feature_reduction = best_model_obj.get_df()
# getCombinations(df_feature_reduction.columns.to_list())

# for feature in list(df_feature_reduction.columns.to_list()):
#     df_feature_reduction.drop(columns=feature)
#     X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#     best_model_obj_copy = best_model_obj.copy()
#     best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

In [137]:
# #Wrapper approaches
# #Backward elimination using Recursive feature elimination¶
# from sklearn.svm import SVC
# from sklearn import feature_selection
# import pandas as pd
# import numpy as np


# x, y = df.drop(columns=["price"]), df["price"]
# # x_y= np.concatenate((x.reshape(150,4),y.reshape(150,1)),1)
# x_y= np.concatenate((x,y),1)
# #print(x_y)
# #print(iris.target_names)

# #Create column list
# df_columns=df.columns
# # df_columns.append("Label")
# # #Create PANDAS data frame
# # df = pd.DataFrame(x_y,columns=df_columns)
# #Map label index to label name
# df['price']=df['price'].map(lambda x: iris.target_names[int(x)])
# print("\nOriginal Iris Data Set:")
# print(df)

# #Create the RFE object and rank features
# num_features=2
# svc = SVC(kernel="linear", C=1)
# rfe = feature_selection.RFE(estimator=svc, n_features_to_select=num_features, step=1)
# rfe.fit(x, y)
# #print("Selected features will have a ranking=1 and support=TRUE")
# #print(iris.feature_names," ",rfe.ranking_," ",rfe.support_)
# #print(x.shape)

# #extend column-mask by one column for the label (always true)
# column_mask=np.append(rfe.support_,True)
# #use list column_mask to mask df-columns list
# reduced_iris_features = [df_columns[i] for i in range(len(df_columns)) if column_mask[i]]
# print("Reduced_iris_features: ",reduced_iris_features)
# # use reduced_iris_features to reduce the data frame
# reduced_df=df[reduced_iris_features]

# print("\nIris Data Set reduced to ",num_features," features: \n",reduced_df)

In [138]:
#TODO sqft_living wenn price auffällig niedrig obwohl m² hoch clsuter
#TODO PCA macht warscheinlich wenig sinn